Compute a flag based on seed clustereing

In [84]:
import os
from pathlib import Path
import numpy as np
import requests
import pandas as pd
import joblib

from sbdata.FlipsideApi import FlipsideApi

In [85]:
# Set path to data folder
current_dir = Path(os.getcwd())
DATA_DIR = os.path.join(current_dir.parent.parent, 'data-regen-rangers')
DATA_DIR_GITCOIN = os.path.join(current_dir.parent.parent, 'data-gitcoin')
GRANT_ROUND_ID = '0x984e29dCB4286c2D9cbAA2c238AfDd8A191Eefbc'
ODC_DATA_DIR = os.path.join(current_dir.parent.parent.parent, 'data')

PATH_TO_VOTES = os.path.join(DATA_DIR_GITCOIN, 'citizen-votes.csv')

In [86]:
df_matching_address = pd.read_csv(f'../output_gitcoin/full_features/voters_features_{GRANT_ROUND_ID}.csv')
df_votes_feature = pd.read_csv(f'../output_gitcoin/full_features/votes_features_citizen_{GRANT_ROUND_ID}.csv')
df_votes = pd.read_csv(PATH_TO_VOTES).drop(columns=['__row_index'])

In [87]:
df_matching_address.columns

Index(['seed_same_naive', 'seed_same', 'seed_suspicious', 'less_5_tx',
       'less_10_tx', 'interacted_other_ctbt', 'lcs', 'cluster_size_lcs',
       'mean_score_lcs', 'max_score_lcs', 'has_lcs',
       'count_interaction_with_pool', 'count_interaction_with_toxic',
       'interact_less_5tx', 'has_interaction_toxic', 'has_no_pool_interaction',
       'count_interaction_with_airdrop_m', 'is_airdrop_master',
       'count_interaction_with_tornado', 'count_interaction_with_disperse',
       'has_interaction_airdrop_m', 'has_interaction_tornado',
       'has_interaction_disperse', 'flagged', 'stakeridoo_detected',
       'doge_detected', 'really_suspicicious_cluster', 'odc_detected',
       'trusta', 'stakeridoo_detected.1', 'doge_detected.1',
       'really_suspicicious_cluster.1', 'odc_detected.1', 'address'],
      dtype='object')

In [88]:
df_votes.columns

Index(['block_timestamp', 'tx_hash', 'voter', 'project', 'amount_usd'], dtype='object')

In [89]:
print(df_matching_address.shape)
print(df_votes_feature.shape)

(17023, 34)
(57366, 33)


Get all the voter addresses

In [90]:
a_voter_addresses = df_votes['voter'].unique()

In [91]:
# 3000 random addresses to prevent heavy querying, it is not randomm for reproducibility
small_a_voter_addresses = a_voter_addresses[3000:6000]

In [92]:
sql_template = """
WITH
  transactions AS (
    SELECT
      BLOCK_TIMESTAMP,
      TX_FEE,
      FROM_ADDRESS AS EOA,
      TO_ADDRESS AS COUNTERPARTY,
      ETH_VALUE,
      1 as BOOLEAN_OUT
    FROM
      optimism.core.fact_transactions
    WHERE
      FROM_ADDRESS IN (%s)
    UNION ALL
    SELECT
      BLOCK_TIMESTAMP,
      TX_FEE,
      TO_ADDRESS AS EOA,
      TO_ADDRESS AS COUNTERPARTY,
      ETH_VALUE,
      0 as BOOLEAN_OUT
    FROM
      optimism.core.fact_transactions
    WHERE
      TO_ADDRESS IN (%s)
  )
SELECT
  EOA,
  COUNT(*) as n_tx,
  COUNT(DISTINCT(COUNTERPARTY)) as n_counterparty,
  SUM(ETH_VALUE) as eth_volume,
  SUM(BOOLEAN_OUT) as n_tx_out,
  n_tx - n_tx_out as n_tx_in,
  n_tx_out - n_tx_in as n_tx_diff_out_in,
  TIMESTAMPDIFF(MINUTE, MIN(BLOCK_TIMESTAMP), CURRENT_TIMESTAMP()) as age,
  TIMESTAMPDIFF(MINUTE, MIN(BLOCK_TIMESTAMP), MAX(BLOCK_TIMESTAMP)) as time_alive,
  CASE WHEN age = 0 THEN 0 ELSE n_tx / age END as tx_min,
  CASE WHEN time_alive = 0 THEN 0 ELSE n_tx / time_alive END as tx_min_alive,
  AVG(TX_FEE) as avg_tx_fee,
  STDDEV(TX_FEE) as std_tx_fee
FROM
  transactions
GROUP BY
  EOA
HAVING
  n_tx < 100;
"""

In [93]:
# set up api key for flipside
api_key = os.environ['FLIPSIDE_API_KEY2']
flipside_api = FlipsideApi(api_key, timeout_minutes=60, max_address=2000)

In [116]:
a_voter_addresses.shape

(17023,)

In [94]:
df_features_add = flipside_api.extract_data_flipside(a_voter_addresses, sql_template)

Extracting for address: 0 - 2000


Extracting for address: 2000 - 4000
Extracting for address: 4000 - 6000
Extracting for address: 6000 - 8000
Extracting for address: 8000 - 10000
Extracting for address: 10000 - 12000
Extracting for address: 12000 - 14000
Extracting for address: 14000 - 16000
Extracting for address: 16000 - 18000


In [95]:
df_features_add['ratio_tx_counterparties'] = df_features_add['n_tx'] / df_features_add['n_counterparty']
df_features_add.drop('__row_index', axis=1, inplace=True)
df_features_add.set_index('eoa', inplace=True)
df_features_add.head(2)

,n_tx,n_counterparty,eth_volume,n_tx_out,n_tx_in,n_tx_diff_out_in,age,time_alive,tx_min,tx_min_alive,avg_tx_fee,std_tx_fee,ratio_tx_counterparties
eoa,,,,,,,,,,,,,
0xb06c8846ba233938605af625d7ce31190ab604bc,86,48,0.338475,83,3,80,278958,264100,0.000308,0.000326,0.000153,0.000144,1.791667
0x4aa023ca41346cc2869ed4db7588fc58055074e9,72,37,0.439118,69,3,66,289407,276076,0.000249,0.000261,0.000200,0.000233,1.945946


In [96]:
df_features_add.isna().sum()

n_tx                        0
n_counterparty              0
eth_volume                  0
n_tx_out                    0
n_tx_in                     0
n_tx_diff_out_in            0
age                         0
time_alive                  0
tx_min                      0
tx_min_alive                0
avg_tx_fee                  0
std_tx_fee                 99
ratio_tx_counterparties     0
dtype: int64

In [97]:
df_features_add.std_tx_fee.fillna(0, inplace=True)

In [98]:
model = joblib.load('optimism_cex_dex_logistic_best.joblib')
prediction = model.predict(df_features_add)

In [114]:
print(df_features_add.shape)
prediction.size

(9333, 14)


9199

In [99]:
df_features_add['prediction'] = prediction

In [100]:
df_features_add['prediction'].value_counts()

prediction
0.0    9065
1.0     134
Name: count, dtype: int64

This does not sum to 100% for some unknown reason, we will explore below which addresses are missing.

In [101]:
df_features_add[df_features_add['prediction']==1].sort_values(by='n_tx', ascending=False).head(10)

,n_tx,n_counterparty,eth_volume,n_tx_out,n_tx_in,n_tx_diff_out_in,age,time_alive,tx_min,tx_min_alive,avg_tx_fee,std_tx_fee,ratio_tx_counterparties,prediction
eoa,,,,,,,,,,,,,,
0x9fc64deb8ba637656b47557acb3e9ed742160541,40,17,0.228527,37,3,34,61877,58788,0.000646,0.000680,0.003297,0.007117,2.352941,1.0
0x164cd0c8124159ebffc74951484616699f74039d,34,16,0.101946,32,2,30,49202,48760,0.000691,0.000697,0.000946,0.002819,2.125000,1.0
0x43d21aa7d4e8027e7a462e41838fc8d258f91f26,23,15,15.953949,19,4,15,22822,31,0.001008,0.741935,0.000063,0.000050,1.533333,1.0
0x4c5b9a96b1b522cf5beed817e76426c2912d06b9,18,18,0.001600,18,0,18,25656,9,0.000702,2.000000,0.000040,0.000028,1.000000,1.0
0x230acfeb57ad3fd38a43437e841c8305fe2c23e7,17,10,0.791224,17,0,17,33353,36,0.000510,0.472222,0.000061,0.000029,1.700000,1.0
0x8f20ca997300ff01df346d5db562759ec2c4cc41,15,7,0.033928,15,0,15,35152,17,0.000427,0.882353,0.000062,0.000037,2.142857,1.0
0x4309b0c83036fb2e169ccc8fe2c5a51b7585493c,15,10,0.000982,15,0,15,25646,15,0.000585,1.000000,0.000045,0.000022,1.500000,1.0
0xcfc25811af4d9073214b5f652fc6530c4adb93de,11,11,0.022030,10,1,9,69478,65246,0.000158,0.000169,0.000888,0.002498,1.000000,1.0
0x94879afb25ea02f7c3736f153b4ce27e5765a05d,8,7,0.013798,7,1,6,92277,63346,0.000087,0.000126,0.000743,0.000647,1.142857,1.0


Lets manually check the first 10 addresses

These wallets look a lot like airdrop farmer wallets. The time alive for some of these wallets is very short indicating a automatic behavior / bot

This legos may be more reliable than others such as aidrop master but we can't be sure of the accuracy of this method.

## Exploring the missing addresses

In [117]:
arr_diff = np.setdiff1d(df_features_add.index, df_matching_address.index.unique())
print(len(arr_diff))

9199


In [118]:
# Lets try to export the data again for these addresses
df_feature_miss = flipside_api.extract_data_flipside(arr_diff, sql_template)

Extracting for address: 0 - 2000
Extracting for address: 2000 - 4000
Extracting for address: 4000 - 6000
Extracting for address: 6000 - 8000
Extracting for address: 8000 - 10000


In [104]:
df_feature_miss.shape

(134, 14)

In [105]:
df_feature_miss

,eoa,n_tx,n_counterparty,eth_volume,n_tx_out,n_tx_in,n_tx_diff_out_in,age,time_alive,tx_min,tx_min_alive,avg_tx_fee,std_tx_fee,__row_index
0,0x94879afb25ea02f7c3736f153b4ce27e5765a05d,8,7,0.013798,7,1,6,92266,63346,0.000087,0.000126,0.000743,0.000647,0
1,0xbfec3946ff5bc73843f48b3be4b14c7fd36f2422,2,2,0.002175,1,1,0,41237,2,0.000049,1.000000,0.000046,0.000025,1
2,0x181337a1c5268a36bb3001e77afd44c2c69786c6,2,2,0.012490,1,1,0,42582,1,0.000047,2.000000,0.000076,0.000030,2
3,0xf41b98a4c32bb61468c8001e0c69ef64ce6dea57,3,3,0.001000,2,1,1,22715,8,0.000132,0.375000,0.000047,0.000031,3
4,0xc457905e836fddd29a731964942271689c1456ed,3,3,0.000537,3,0,3,23115,1,0.000130,3.000000,0.000191,0.000082,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0x425d84e63e623d0ad74d324710e5d0c53656ca7a,3,2,0.049011,2,1,1,24360,7,0.000123,0.428571,0.000078,0.000020,129
130,0x5b5d6d40da8eb176e8329e328463aac06a5309a2,2,2,0.001384,1,1,0,26192,1,0.000076,2.000000,0.000038,0.000019,130
131,0x2717ae2060084918a338b3a0b3a216cedd810aca,2,2,0.003071,1,1,0,35297,2,0.000057,1.000000,0.000052,0.000040,131
132,0x8073639b11994c549eda58fc3cd7132a72aadf10,1,1,0.073000,1,0,1,43223,0,0.000023,0.000000,0.000808,NaN,132


This may because of the flipside api with too many rows in memmory at some point in time. We should use max_address = 3000

In [106]:
df_feature_miss['ratio_tx_counterparties'] = df_feature_miss['n_tx'] / df_feature_miss['n_counterparty']
df_feature_miss.drop('__row_index', axis=1, inplace=True)
df_feature_miss.set_index('eoa', inplace=True)

In [107]:
df_features_add = pd.concat([df_features_add, df_feature_miss])
df_features_add['prediction'].value_counts()

prediction
0.0    9065
1.0     134
Name: count, dtype: int64

In [108]:
df_features_add.tail(20)

,n_tx,n_counterparty,eth_volume,n_tx_out,n_tx_in,n_tx_diff_out_in,age,time_alive,tx_min,tx_min_alive,avg_tx_fee,std_tx_fee,ratio_tx_counterparties,prediction
eoa,,,,,,,,,,,,,,
0x96178dacb4c441efd369a09b5bbb0dace326c7c5,3,3,0.000530,3,0,3,23000,1,0.000130,3.000000,0.000067,0.000027,1.000000,NaN
0xf48ce8563f180792352c2e833e2c0746a693a2d5,2,2,0.006050,1,1,0,24460,6,0.000082,0.333333,0.000102,0.000097,1.000000,NaN
0x14acdf57dd0f21acfca9e9b0ca2b9b9d3aba285b,3,3,0.001940,3,0,3,23183,10,0.000129,0.300000,0.000111,0.000119,1.000000,NaN
0x9e394444243f8fb893cf364ffe4df4ba02ed3a3b,2,2,0.007459,1,1,0,24749,7,0.000081,0.285714,0.000068,0.000058,1.000000,NaN
0x958fa668369d42afccd1c255195bb1ef2fe05ef4,5,3,0.007567,4,1,3,25833,12,0.000194,0.416667,0.000042,0.000011,1.666667,NaN
0xb9fabfaf84a94ac57b2a429d76d277fff83dd3c8,2,2,0.018767,1,1,0,30700,6,0.000065,0.333333,0.000100,0.000044,1.000000,NaN
0x93481c2b2daf2dc77c44504406e28f33c1ffa681,4,4,0.002796,3,1,2,25737,9,0.000155,0.444444,0.000060,0.000017,1.000000,NaN
0x712b4018ce224dca5ca9100f63c14893f977167d,2,2,0.004064,1,1,0,25130,2,0.000080,1.000000,0.000062,0.000004,1.000000,NaN
0x6182c4b83d2d0de73bf124aaa45a6ef52ba7cb4b,2,2,0.067367,1,1,0,24813,3,0.000081,0.666667,0.000202,0.000188,1.000000,NaN


So the real reason is that many addresses are nan out of the model

In [109]:
df_features_add.shape

(9333, 14)

In [110]:
len(a_voter_addresses)

17023